In [1]:
!nvidia-smi

Wed Jul  7 15:19:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 471.11       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   56C    P0    25W /  N/A |    815MiB /  6144MiB |    ERR!      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import os
import pandas as pd

import sys
sys.path.append('..')

from deepmatcher.data.custom_dataset import DeepMatcherDataset
from deepmatcher.data.encoder.text_encoders import HFTextEncoder
from deepmatcher.optim import Optimizer

from torch.utils.data import DataLoader
from deepmatcher.models.core import MatchingModel

In [3]:
dataset_dir = "sample_data/itunes-amazon/"
train_df = pd.read_csv(os.path.join(dataset_dir, 'train.csv'))
print('train:', train_df.shape)
val_df = pd.read_csv(os.path.join(dataset_dir, 'validation.csv'))
print('val:', val_df.shape)
test_df = pd.read_csv(os.path.join(dataset_dir, 'train.csv'))
print('test:', test_df.shape)

train_df.head()

train: (323, 18)
val: (108, 18)
test: (323, 18)


,id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
0,448,0,Baby When the Light ( David Guetta & Fred Rist...,David Guetta,Pop Life ( Extended Version ) [ Bonus Version ],"Dance , Music , Rock , Pop , House , Electroni...",$ 1.29,‰ ãÑ 2007 Gum Records,6:17,18-Sep-07,Revolver ( Madonna Vs. David Guetta Feat . Lil...,David Guetta,One Love ( Deluxe Version ),Dance & Electronic,$ 1.29,( C ) 2014 Swedish House Mafia Holdings Ltd ( ...,3:18,"August 21 , 2009"
1,287,1,Outversion,Mark Ronson,Version,"Pop , Music , R&B / Soul,Soul,Dance,Rock,Jazz,...",$ 0.99,2007 Mark Ronson under exclusive license to SO...,1:50,10-Jul-07,Outversion,Mark Ronson,Version [ Explicit ],Pop,$ 0.99,( c ) 2011 J'adore Records,1:50,"July 10 , 2007"
2,534,0,Peer Pressure ( feat . Traci Nelson ),Snoop Dogg,Doggumentary,"Hip-Hop/Rap , Music , Rock , Gangsta Rap , Wes...",$ 1.29,"‰ ãÑ 2011 Capitol Records , LLC . All rights r...",4:07,29-Mar-11,Boom ( ( Feat . T-Pain ) [ Edited ] ),Snoop Dogg,Doggumentary [ Edited ],"Rap & Hip-Hop , West Coast",$ 1.29,"( C ) 2011 Capitol Records , LLC",3:50,"March 29 , 2011"
3,181,1,Stars Come Out ( Tim Mason Remix ),Zedd,Stars Come Out ( Remixes ) - EP,"Dance , Music , Electronic , House",$ 1.29,2012 Dim Mak Inc.,5:49,20-May-14,Stars Come Out ( Dillon Francis Remix ),Zedd,Stars Come Out [ Dillon Francis Remix ],Dance & Electronic,$ 1.29,2012 Dim Mak Inc.,4:08,"May 20 , 2014"
4,485,0,Jump ( feat . Nelly Furtado ),Flo Rida,R.O.O.T.S. ( Deluxe Version ),"Hip-Hop/Rap , Music",$ 1.29,‰ ãÑ 2009 Atlantic Recording Corporation for t...,3:28,30-Mar-09,"Yayo [ Feat . Brisco , Billy Blue , Ball Greez...",Flo Rida,R.O.O.T.S. ( Route Of Overcoming The Struggle ...,Rap & Hip-Hop,$ 1.29,"( C ) 2012 Motown Records , a Division of UMG ...",7:53,"March 30 , 2009"


In [4]:
text_transformer = HFTextEncoder('sentence-transformers/paraphrase-albert-small-v2', trainable=True, seqtovec='mean')

In [5]:
label_column = 'label'
text_columns = ['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released']
image_col = ''
tokenizer = text_transformer.tokenizer

def get_dataloader(data_df, label_column, text_cols, imaeg_col, tokenizer, batch_size, shuffle, num_workers):
    dataset = DeepMatcherDataset(
        data_df=data_df,
        label_col=label_column,
        text_cols=text_columns,
        image_col=image_col,
        tokenizer=tokenizer
        )
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=dataset.collate_fn)

batch_size = 16
num_workers = 4

train_dataloader = get_dataloader(train_df, label_column, text_columns, image_col, tokenizer, batch_size, True, num_workers)
val_dataloader = get_dataloader(val_df, label_column, text_columns, image_col, tokenizer, batch_size, False, num_workers)
test_dataloader = get_dataloader(test_df, label_column, text_columns, image_col, tokenizer, batch_size, False, num_workers)

received data_df : (323, 18)
supplied text columns : Song_Name, Artist_Name, Album_Name, Genre, Price, CopyRight, Time, Released
supplied image column : 
supplied label column : label
received data_df : (108, 18)
supplied text columns : Song_Name, Artist_Name, Album_Name, Genre, Price, CopyRight, Time, Released
supplied image column : 
supplied label column : label
received data_df : (323, 18)
supplied text columns : Song_Name, Artist_Name, Album_Name, Genre, Price, CopyRight, Time, Released
supplied image column : 
supplied label column : label


In [6]:
model = MatchingModel(
    text_encoder=text_transformer,
    image_encoder=None,
    attr_summarizer=None,
    attr_condense_factor='auto',
    attr_comparator='concat',
    attr_merge='concat',
    classifier='2-layer-highway',
    hidden_size=300,
    text_attrs=text_columns,
    image_attr=None
    )

text_attrs: ['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released']
image_attr: None
prefixes: ('left_', 'right_')
attrs: ['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released']
all attrs: ['left_Song_Name', 'right_Song_Name', 'left_Artist_Name', 'right_Artist_Name', 'left_Album_Name', 'right_Album_Name', 'left_Genre', 'right_Genre', 'left_Price', 'right_Price', 'left_CopyRight', 'right_CopyRight', 'left_Time', 'right_Time', 'left_Released', 'right_Released']


In [7]:
model.initialize(next(iter(train_dataloader))['attrs'])
model

/home/sour4bh/.local/lib/python3.8/site-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


MatchingModel(
  (text_encoder): HFTextEncoder(
    (model): AlbertModel(
      (embeddings): AlbertEmbeddings(
        (word_embeddings): Embedding(30000, 128, padding_idx=0)
        (position_embeddings): Embedding(512, 128)
        (token_type_embeddings): Embedding(2, 128)
        (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0, inplace=False)
      )
      (encoder): AlbertTransformer(
        (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
        (albert_layer_groups): ModuleList(
          (0): AlbertLayerGroup(
            (albert_layers): ModuleList(
              (0): AlbertLayer(
                (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
                (attention): AlbertAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                

In [8]:
optimizer = Optimizer(lr=1e-3, lr_decay=1e-1, start_decay_at=3)

In [9]:
model.run_train(
    train_dataloader,
    val_dataloader,
    best_save_path='best_mode.pth',
    epochs=15,
    )

* Number of trainable parameters: 12960586
===>  TRAIN Epoch 1


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 4.46 GiB already allocated; 0 bytes free; 4.68 GiB reserved in total by PyTorch)

In [ ]:
model.run_eval(test_dataset)